In [1]:
# Dependencies
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

In [8]:
wildfire_df = pd.read_csv('NOAA_csvs/wildfires_2010_2019.csv')
combo = pd.read_csv('NOAA_csvs/wf_combined_zone_county.csv')

In [11]:
# Drop undated fires.
combo.dropna(subset=['Date','County'],inplace=True)
# Group by NOAA FIP
combo.groupby("NOAA_FIP")
combo_counts = pd.DataFrame(combo["County"].value_counts())
combo_counts = combo_counts.reset_index()
combo_counts.rename(columns={"index":"County","County":"Fires"},inplace=True)
combo_counts = combo_counts.sort_values("Fires",ascending=False)
combo_counts.set_index("County")
combo_counts.reset_index(inplace=True)
combo_counts.drop(columns=["index"],inplace=True)
combo_counts

,County,Fires
0,San Bernardino,47
1,Riverside,45
2,Siskiyou,40
3,San Diego,30
4,Los Angeles,22
5,Ventura,19
6,Modoc,19
7,Nevada,19
8,Yuba,18
9,Placer,17


In [7]:
def rename_location(fire):
    name = fire.partition(" COUNTY")
    fire = str(name[0] + name[1])
    return fire
def filter_counties(list1, list2):
    return ([i for i in list1 if
             any(j in i for j in list2)])

In [15]:
# Variables for parsing lists.
fire = ""
new_names = []
chosen_names = []
substring = '(ZONE)'
county_names = [wildfire_df['CZ_NAME_STR']]

# Parses through both lists of counties and pulls out only shared values.
for fire in wildfire_df['CZ_NAME_STR']:
    fire = rename_location(fire)
    new_names.append(fire)
wildfire_df["CZ_NAME_STR"] = new_names
for name in new_names:
    if substring not in name:
        if name not in chosen_names:
            chosen_names.append(name.title())
for name in wildfire_df['CZ_NAME_STR']:
    county_names.append(name.title())
list1 = county_names
list2 = chosen_names
utile_counties = filter_counties(list1, list2)
for county in range(len(utile_counties)):
    utile_counties[county] = utile_counties[county].upper()

In [17]:
# Reduces NOAA data to rows found in the usable counties. (119 Fires in Ten Years)
clean_wf = pd.DataFrame()
clean_wf = wildfire_df[wildfire_df["CZ_NAME_STR"].isin(utile_counties)]
# Renames columns.
clean_wf = clean_wf[['EVENT_ID', 'CZ_NAME_STR', 'BEGIN_DATE', 'EVENT_TYPE', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 
                                'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DAMAGE_PROPERTY_NUM', 'DAMAGE_CROPS_NUM', 'CZ_FIPS']]
clean_wf.rename(columns={'EVENT_ID': 'NOAA ID', 
                                'CZ_NAME_STR': 'County Name', 
                                'BEGIN_DATE': 'Date', 
                                'EVENT_TYPE': 'Type', 
                                'DEATHS_DIRECT': 'Direct Deaths', 
                                'DEATHS_INDIRECT': 'Indirect Deaths', 
                                'INJURIES_DIRECT': 'Direct Injuries', 
                                'INJURIES_INDIRECT': 'Indirect Injuries', 
                                'DAMAGE_PROPERTY_NUM': 'Property Damage', 
                                'DAMAGE_CROPS_NUM': 'Crop Damage', 
                                'CZ_FIPS': 'NOAA_FIP'}, inplace=True)
clean_wf

,NOAA ID,County Name,Date,Type,Direct Deaths,Indirect Deaths,Direct Injuries,Indirect Injuries,Property Damage,Crop Damage,NOAA_FIP
0,234746,SAN BERNARDINO AND RIVERSIDE COUNTY,05/20/2010,Wildfire,0,0,0,1,0,0,48
1,250167,SAN DIEGO COUNTY,07/13/2010,Wildfire,0,0,0,0,0,0,43
2,250175,SAN BERNARDINO AND RIVERSIDE COUNTY,07/15/2010,Wildfire,0,0,1,0,17500,0,48
3,250169,RIVERSIDE COUNTY,07/15/2010,Wildfire,0,0,0,0,0,0,56
4,250171,SAN BERNARDINO AND RIVERSIDE COUNTY,07/15/2010,Wildfire,0,0,0,0,0,0,48
...,...,...,...,...,...,...,...,...,...,...,...
392,861476,VENTURA COUNTY,10/31/2019,Wildfire,0,0,0,0,0,0,45
393,863910,VENTURA COUNTY,11/01/2019,Wildfire,0,0,0,0,0,0,45
394,863913,VENTURA COUNTY,11/01/2019,Wildfire,0,0,0,0,0,0,44
395,863914,VENTURA COUNTY,11/01/2019,Wildfire,0,0,0,0,0,0,45


In [21]:
clean_wf['County Name'] = clean_wf['County Name'].title()

AttributeError: 'Series' object has no attribute 'title'